In [27]:
import pandas as pd
from imblearn.over_sampling import ADASYN
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder


# WORK IN PROGRESS DO NOT USE

In [28]:
# First step is to load our data
dataFrame = pd.read_csv('data/Week_5_NSL-KDD-Dataset/kdd_merged.csv')

# Indices of nominal features
nominal_columns = dataFrame.columns[[1, 2, 3, 41, 43]].tolist()
binary_columns = dataFrame.columns[[6, 11, 13, 14, 20, 21]].tolist()
numeric_columns = [col for col in dataFrame.columns if col not in nominal_columns and col not in binary_columns]

# Remove useless item
dataFrame.drop('num_outbound_cmds', inplace=True, axis=1)
numeric_columns.remove('num_outbound_cmds')

# Fix SU Attempted
dataFrame['su_attempted'] = dataFrame['su_attempted'].replace(2, 0)
# nominal_columns, binary_columns, numeric_columns

# 1. Extract features and target variable

X = dataFrame.drop(['attack_category', 'attack_type'], axis=1)
nominal_columns.remove('attack_category')
nominal_columns.remove('attack_type')

y = dataFrame['attack_category']


In [29]:
# 3. Scale numeric features - Identify and handle non-numeric columns
scaler = StandardScaler()
# Get numeric columns excluding those with non-numeric data
X[numeric_columns] = scaler.fit_transform(X[numeric_columns])  # Apply scaling to numeric columns only

# 4. Create dummy variables for nominal and binary features
columns_to_encode = nominal_columns

X = pd.get_dummies(X, columns=columns_to_encode, drop_first=True)  #drop_first to avoid dummy variable trap

In [30]:
# 5. Split data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 6. Apply ADASYN (if needed)
adasyn = ADASYN(sampling_strategy='auto', random_state=42)
X_train, y_train = adasyn.fit_resample(X_train, y_train)

In [31]:
import numpy as np
from sklearn.metrics import zero_one_loss, accuracy_score, classification_report, confusion_matrix
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis

def print_score(classifer_name: str, y_test_data, y_pred_data):
    print(f"\n{classifer_name}\n")
    print(f"Zero One Loss: {zero_one_loss(y_test_data, y_pred_data):.4f}")
    print(f"Accuracy: {accuracy_score(y_test_data, y_pred_data):.2f}")
    print("\nClassification Report:\n", classification_report(y_test_data, y_pred_data))
    print("\nConfusion Matrix:\n", confusion_matrix(y_test_data, y_pred_data))
    # print("\nR2 Score:\n", calculate_r2_score(y_test_data, y_pred_data))

print("QDA: Cleaning Correleation Matrix:")
# Removing Highly correlated items.
# Compute correlation matrix
correlation_matrix = X_train.corr().abs()

# Select upper triangle of the correlation matrix
upper_tri = correlation_matrix.where(np.triu(np.ones(correlation_matrix.shape), k=1).astype(bool))

# Find features with high correlation
to_drop = [column for column in upper_tri.columns if any(upper_tri[column] > 0.9)]

print(f"Removing Features: {to_drop}")

# Drop from both train and test sets
X_train_resampled = X_train.drop(columns=to_drop)
X_test = X_test.drop(columns=to_drop)

qda = QuadraticDiscriminantAnalysis(reg_param=0.1)
qda.fit(X_train_resampled, y_train)
y_pred = qda.predict(X_test)
print_score("QDA", y_test, y_pred)

QDA: Cleaning Correleation Matrix:
Removing Features: ['num_root', 'srv_serror_rate', 'srv_rerror_rate', 'dst_host_serror_rate', 'dst_host_srv_serror_rate', 'flag_S0']


C:\Users\shoun\miniconda3\Lib\site-packages\sklearn\discriminant_analysis.py:947: UserWarning: Variables are collinear
  warnings.warn("Variables are collinear")



QDA

Zero One Loss: 0.0809
Accuracy: 0.92

Classification Report:
               precision    recall  f1-score   support

      benign       0.99      0.91      0.95     15450
         dos       0.89      0.99      0.94     10723
       probe       0.84      0.83      0.84      2750
         r2l       0.52      0.34      0.41       724
         u2r       0.08      0.49      0.14        57

    accuracy                           0.92     29704
   macro avg       0.67      0.71      0.66     29704
weighted avg       0.93      0.92      0.92     29704


Confusion Matrix:
 [[14107   708   363   213    59]
 [   61 10627    31     4     0]
 [   48   403  2296     0     3]
 [   56   150    23   243   252]
 [    0     9    17     3    28]]
